# `nu18` vs `n24`

`n24` is the "Number of children eligible for Child Tax Credit" according to [Tax-Calculator documentation](http://open-source-economics.github.io/Tax-Calculator/). `nu18` is the number of people under age 18.

Based on the Child Tax Credit's [definition of child](https://www.thebalance.com/child-tax-credit-3193009), this should be a subset of children under age 18. One criterion is being under age 17, and others limit further within that.

This notebook examines tax units that violate this assumption by having `n24 > nu18`. This is discussed in [taxdata issue #157](https://github.com/open-source-economics/taxdata/issues/157).

*Data: CPS  |  Tax year: 2014  |  Author: Max Ghenis*

## Setup

### Imports

In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
tc.__version__

'1.0.1'

In [3]:
# Specify number of decimals in tables.
pd.set_option('precision', 2)

## Summaries

In [4]:
recs = tc.Records.cps_constructor()
calc = tc.Calculator(records=recs, policy=tc.Policy())
calc.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


We only care about records `n24 > 0`.

In [5]:
full = calc.dataframe(['s006', 'nu18', 'n24'])

In [6]:
full['nu18_s006'] = full.nu18 * full.s006
full['n24_s006'] = full.n24 * full.s006
full['n24_gt_nu18'] = full.n24 > full.nu18
full['n24_vs_nu18'] = np.where(full.n24 > full.nu18, 'n24 greater',
                               np.where(full.nu18 > full.n24, 
                                        'nu18 greater', 'equal'))

Total `nu18` and `n24`.

In [7]:
full.n24_s006.sum() / 1e6

64.33535522000001

In [8]:
full.nu18_s006.sum() / 1e6

82.40831775000001

In [9]:
full.pivot_table(index='n24_vs_nu18', 
                 values=['s006', 'n24_s006', 'nu18_s006'],
                 aggfunc=sum)

,n24_s006,nu18_s006,s006
n24_vs_nu18,,,
equal,5.96e+07,5.96e+07,1.36e+08
nu18 greater,4.78e+06,2.28e+07,1.32e+07


Limit to `n24>0` for remainder.

In [10]:
df = full[full.n24 > 0]

In [11]:
df.pivot_table(index='n24_gt_nu18', 
               values=['s006', 'n24_s006', 'nu18_s006'],
               aggfunc=sum)

,n24_s006,nu18_s006,s006
n24_gt_nu18,,,
False,6.43e+07,6.75e+07,3.57e+07


Drill into records where `n24>nu18` by `nu18`.

In [12]:
df[df.n24_gt_nu18].pivot_table(
    index='nu18', 
    values=['s006', 'n24_s006', 'nu18_s006'],
    aggfunc=sum)

""
nu18


In [13]:
df[df.n24_gt_nu18].pivot_table(
    index='nu18', 
    columns='n24',
    values=['s006', 'n24_s006', 'nu18_s006'],
    aggfunc=sum)

nu18


Share of tax units with `n24>0` where `n24>nu18`.

In [14]:
df.loc[df.n24_gt_nu18, 's006'].sum() / df.s006.sum()

0.0